In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Sergipe - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Consumo de Cimento (t)
0,2003,1868.137980,0.669217,8.389943,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,4.173873,2.159732e+07,1.260978e+06,9.960678,2.029642e+07,245.311000
1,2004,1928.892034,0.673124,8.319423,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,4.300620,2.188783e+07,1.274840e+06,9.976013,2.036790e+07,222.063000
2,2005,2088.926497,0.676629,8.294120,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,4.260280,2.214600e+07,1.279358e+06,9.984663,2.041275e+07,221.797000
3,2006,2366.448369,0.679740,8.204415,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,4.183292,2.238826e+07,1.291619e+06,9.998498,2.046011e+07,270.225000
4,2007,2563.277811,0.682466,8.179910,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,4.227134,2.258998e+07,1.301865e+06,10.000737,2.047658e+07,319.406000
5,2008,2685.359823,0.684816,8.151229,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,4.333995,2.275243e+07,1.304488e+06,9.995682,2.046863e+07,391.414000
6,2009,2836.873530,0.686798,8.181780,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,4.469789,2.284202e+07,1.307162e+06,9.962429,2.041487e+07,404.787000
7,2010,2982.517993,0.688421,8.145498,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,4.706852,2.337741e+07,1.316094e+06,9.971938,2.052469e+07,480.282000
8,2011,3066.509797,0.689694,8.132819,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,4.918522,2.381828e+07,1.308563e+06,9.960908,2.056187e+07,579.482000
9,2012,3182.329234,0.690538,11.832646,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,5.074864,2.420222e+07,1.297619e+06,9.936855,2.057944e+07,646.084000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
0,1868.137980,0.669217,8.389943,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,4.173873,2.159732e+07,1.260978e+06,9.960678,2.029642e+07
1,1928.892034,0.673124,8.319423,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,4.300620,2.188783e+07,1.274840e+06,9.976013,2.036790e+07
2,2088.926497,0.676629,8.294120,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,4.260280,2.214600e+07,1.279358e+06,9.984663,2.041275e+07
3,2366.448369,0.679740,8.204415,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,4.183292,2.238826e+07,1.291619e+06,9.998498,2.046011e+07
4,2563.277811,0.682466,8.179910,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,4.227134,2.258998e+07,1.301865e+06,10.000737,2.047658e+07
5,2685.359823,0.684816,8.151229,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,4.333995,2.275243e+07,1.304488e+06,9.995682,2.046863e+07
6,2836.873530,0.686798,8.181780,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,4.469789,2.284202e+07,1.307162e+06,9.962429,2.041487e+07
7,2982.517993,0.688421,8.145498,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,4.706852,2.337741e+07,1.316094e+06,9.971938,2.052469e+07
8,3066.509797,0.689694,8.132819,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,4.918522,2.381828e+07,1.308563e+06,9.960908,2.056187e+07
9,3182.329234,0.690538,11.832646,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,5.074864,2.420222e+07,1.297619e+06,9.936855,2.057944e+07


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     222.063000
1     221.797000
2     270.225000
3     319.406000
4     391.414000
5     404.787000
6     480.282000
7     579.482000
8     646.084000
9     590.339000
10    633.190143
11    592.014514
12    552.624343
13    473.789000
14    372.210000
15    409.903000
16    526.396000
17    511.494000
18    425.370000
19           NaN
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
0,-1.516349,-2.032630,-0.861098,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.228468,-1.698266,-0.153650,0.587431,-2.389752
1,-1.380665,-1.610601,-0.888811,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.025356,-1.452606,0.200511,0.808511,-1.357062
2,-1.023254,-1.232044,-0.898754,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.090001,-1.234298,0.315945,0.933226,-0.709137
3,-0.403454,-0.896014,-0.934006,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.213374,-1.029436,0.629203,1.132695,-0.024983
4,0.036132,-0.601572,-0.943636,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-1.143117,-0.858859,0.890959,1.164968,0.212977
5,0.308782,-0.347775,-0.954906,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.971871,-0.721495,0.958000,1.092096,0.098136
6,0.647163,-0.133681,-0.942901,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.754259,-0.645730,1.026295,0.612664,-0.678551
7,0.972436,0.041652,-0.957158,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.374363,-0.193003,1.254506,0.749766,0.908040
8,1.160018,0.179166,-0.962141,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.035160,0.179806,1.062103,0.590739,1.445153
9,1.418682,0.270335,0.491785,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.215380,0.504468,0.782486,0.243962,1.698990


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     222.063000
1     221.797000
2     270.225000
3     319.406000
4     391.414000
5     404.787000
6     480.282000
7     579.482000
8     646.084000
9     590.339000
10    633.190143
11    592.014514
12    552.624343
13    473.789000
14    372.210000
15    409.903000
16    526.396000
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
17,-0.858351,1.1315,1.051443,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.652049,0.766714,-1.279722,-1.105281,-0.602615


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    511.494
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [14]:
train_input.iloc[-(len(train_input)//5):]

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
14,-0.895535,1.340840,1.235574,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.054301,1.158398,-1.563800,-1.410142,-0.241420
15,-1.084398,1.355072,1.201616,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,0.871654,1.123845,-1.764814,-1.309447,-0.291402
16,-0.952320,1.262586,1.154323,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.133307,1.026290,-1.684452,-1.194962,-0.243004


In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2036993244, 1362982108, 1825883455, 2519891286, 618679037, 409944299, 2827966888, 1415882301, 2916729955, 1204856631, 3158324571, 3612092620, 3027194897, 265142701, 280247028, 212566781, 3273063517, 2223523116, 3811779923, 917532594, 2950636689, 4160988354, 3784677481, 992427773, 750549362, 516556618, 386366588, 1095331337, 3191884051, 715740620, 1214931027, 3937856419, 2095370541, 3164782459, 1203464678, 1126420512, 3259567579, 1502005367, 1710989366, 3084254559, 3587670120, 4211392445, 2580883398, 749821100, 2042253444, 198500346, 2306800136, 1700924811, 3192264045, 3099907240, 3646108124, 1860462448, 1739271550, 1654659947, 3547422686, 2321757409, 3492688085, 1974312967, 4135659273, 2925339236, 41456004, 2398307129, 1757358038, 949234330, 286306286, 709725444, 390134578, 782927034, 2413098842, 3015420659, 691482814, 2996574422, 984934555, 1947762137, 707186953, 964004781, 3102835643, 1947089369, 2664565534, 1533363822, 1783405282, 4056136643, 4053050868, 3914969989, 980723644, 2227

val_loss: 8946.98046875


Step: 86 ___________________________________________
val_loss: 4381.45654296875


Step: 87 ___________________________________________
val_loss: 5757.97998046875


Step: 88 ___________________________________________
val_loss: 4002.057861328125


Step: 89 ___________________________________________
val_loss: 4307.3076171875


Step: 90 ___________________________________________
val_loss: 4312.498046875


Step: 91 ___________________________________________
val_loss: 3741.9931640625


Step: 92 ___________________________________________
val_loss: 4307.19189453125


Step: 93 ___________________________________________
val_loss: 4307.19140625


Step: 94 ___________________________________________
val_loss: 3890.153564453125


Step: 95 ___________________________________________
val_loss: 4645.953125


Step: 96 ___________________________________________
val_loss: 4307.50390625


Step: 97 ___________________________________________
val_loss: 4403.09521484375


Ste

In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 453ms/step - loss: 222754.5469 - val_loss: 190776.6875
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 210190.7969 - val_loss: 52707.2383
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 236087.9375 - val_loss: 142431.5156
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 192235.4688 - val_loss: 85890.8125
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 173670.2344 - val_loss: 116793.0547
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 151745.2188 - val_loss: 94889.7422
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 119426.0625 - val_loss: 120158.4062
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 95608.2344 - val_loss: 44520.0156
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 97528.8984 - val_loss: 46963.1211
Epoch 10/10000
1

Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 34069.4844 - val_loss: 45554.7695
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 33582.8672 - val_loss: 32882.5781
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 30608.9648 - val_loss: 37998.1523
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 40585.5742 - val_loss: 29922.7168
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 37087.6914 - val_loss: 29015.1426
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 36376.1406 - val_loss: 20085.9453
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 34458.6523 - val_loss: 25258.6875
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 30641.9766 - val_loss: 19568.3770
Epoch 84/10000
1/1 [==============================] - 0s 13ms/step - loss: 29953.4824 - val_loss: 23588.7285
Epoch 85/10000
1/1 

Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 16833.7031 - val_loss: 13729.6406
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 15236.5625 - val_loss: 12578.7529
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 15820.1045 - val_loss: 12143.2461
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 13846.5527 - val_loss: 11092.4775
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 11290.6328 - val_loss: 10604.8877
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 11896.8350 - val_loss: 7183.7749
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 9830.5830 - val_loss: 7623.5176
Epoch 158/10000
1/1 [==============================] - 0s 14ms/step - loss: 12291.7109 - val_loss: 17889.5645
Epoch 159/10000
1/1 [==============================] - 0s 14ms/step - loss: 16693.6465 - val_loss: 10124.8115
Epoch 160/100

1/1 [==============================] - 0s 14ms/step - loss: 11232.0186 - val_loss: 7376.2212
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 12044.0752 - val_loss: 7733.9023
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 11645.9971 - val_loss: 7159.8081
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 10530.1943 - val_loss: 7128.4956
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 10394.8984 - val_loss: 7760.0854
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 10097.8340 - val_loss: 7673.5156
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 10229.7148 - val_loss: 8524.9336
Epoch 233/10000
1/1 [==============================] - 0s 13ms/step - loss: 8949.5400 - val_loss: 8211.0381
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 8771.5820 - val_loss: 7697.7109
Epoch 235/10000
1/1 [================

Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 5495.3931 - val_loss: 4353.1919
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 5211.4458 - val_loss: 4311.8950
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 4138.5278 - val_loss: 4421.6157
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 5084.3354 - val_loss: 4312.6172
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 3488.3181 - val_loss: 4239.8394
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 5329.6343 - val_loss: 4308.8994
Epoch 308/10000
1/1 [==============================] - 0s 13ms/step - loss: 4781.2539 - val_loss: 4567.3540
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 4887.8535 - val_loss: 6216.9062
Epoch 310/10000
1/1 [==============================] - 0s 13ms/step - loss: 4199.1987 - val_loss: 6186.5547
Epoch 311/10000
1/1 [=======

...

1/1 [==============================] - 0s 14ms/step - loss: 5678.9658 - val_loss: 7166.5581
Epoch 1287/10000
1/1 [==============================] - 0s 13ms/step - loss: 5676.4258 - val_loss: 7130.6294
Epoch 1288/10000
1/1 [==============================] - 0s 13ms/step - loss: 6378.2578 - val_loss: 8057.2729
Epoch 1289/10000
1/1 [==============================] - 0s 13ms/step - loss: 5739.3618 - val_loss: 6953.9673
Epoch 1290/10000
1/1 [==============================] - 0s 13ms/step - loss: 5678.4072 - val_loss: 7163.8237
Epoch 1291/10000
1/1 [==============================] - 0s 13ms/step - loss: 5691.5259 - val_loss: 7181.7798
Epoch 1292/10000
1/1 [==============================] - 0s 14ms/step - loss: 5674.5234 - val_loss: 7116.0581
Epoch 1293/10000
1/1 [==============================] - 0s 14ms/step - loss: 6187.1665 - val_loss: 6453.1738
Epoch 1294/10000
1/1 [==============================] - 0s 13ms/step - loss: 5716.3301 - val_loss: 7286.7280
Epoch 1295/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 5812.3423 - val_loss: 7556.4829
Epoch 1362/10000
1/1 [==============================] - 0s 13ms/step - loss: 5810.0547 - val_loss: 7547.7031
Epoch 1363/10000
1/1 [==============================] - 0s 14ms/step - loss: 5807.8398 - val_loss: 7539.0093
Epoch 1364/10000
1/1 [==============================] - 0s 13ms/step - loss: 5805.6943 - val_loss: 7530.3926
Epoch 1365/10000
1/1 [==============================] - 0s 14ms/step - loss: 5803.6157 - val_loss: 7521.8813
Epoch 1366/10000
1/1 [==============================] - 0s 14ms/step - loss: 5801.5996 - val_loss: 7513.4604
Epoch 1367/10000
1/1 [==============================] - 0s 14ms/step - loss: 5799.6450 - val_loss: 7505.1255
Epoch 1368/10000
1/1 [==============================] - 0s 14ms/step - loss: 5797.7476 - val_loss: 7496.8926
Epoch 1369/10000
1/1 [==============================] - 0s 13ms/step - loss: 5795.9072 - val_loss: 7488.7456
Epoch 1370/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 5832.3457 - val_loss: 7289.4707
Epoch 1437/10000
1/1 [==============================] - 0s 13ms/step - loss: 5796.0898 - val_loss: 7118.8823
Epoch 1438/10000
1/1 [==============================] - 0s 13ms/step - loss: 5793.2217 - val_loss: 7148.3813
Epoch 1439/10000
1/1 [==============================] - 0s 13ms/step - loss: 5791.8066 - val_loss: 7137.0874
Epoch 1440/10000
1/1 [==============================] - 0s 14ms/step - loss: 5790.4814 - val_loss: 7134.3345
Epoch 1441/10000
1/1 [==============================] - 0s 13ms/step - loss: 5789.1909 - val_loss: 7129.9136
Epoch 1442/10000
1/1 [==============================] - 0s 14ms/step - loss: 5787.9287 - val_loss: 7125.9395
Epoch 1443/10000
1/1 [==============================] - 0s 14ms/step - loss: 5796.1113 - val_loss: 7144.8237
Epoch 1444/10000
1/1 [==============================] - 0s 13ms/step - loss: 5785.4614 - val_loss: 7112.4780
Epoch 1445/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 5758.7100 - val_loss: 7184.1108
Epoch 1512/10000
1/1 [==============================] - 0s 14ms/step - loss: 5757.8853 - val_loss: 7180.1997
Epoch 1513/10000
1/1 [==============================] - 0s 13ms/step - loss: 5757.1064 - val_loss: 7176.4258
Epoch 1514/10000
1/1 [==============================] - 0s 13ms/step - loss: 5756.3696 - val_loss: 7172.7593
Epoch 1515/10000
1/1 [==============================] - 0s 13ms/step - loss: 5755.6680 - val_loss: 7169.2090
Epoch 1516/10000
1/1 [==============================] - 0s 13ms/step - loss: 5755.0029 - val_loss: 7165.7612
Epoch 1517/10000
1/1 [==============================] - 0s 14ms/step - loss: 5758.1733 - val_loss: 7250.5405
Epoch 1518/10000
1/1 [==============================] - 0s 13ms/step - loss: 5755.5317 - val_loss: 7224.9473
Epoch 1519/10000
1/1 [==============================] - 0s 14ms/step - loss: 5753.2549 - val_loss: 7135.2012
Epoch 1520/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 6286.6997 - val_loss: 6212.9780
Epoch 1587/10000
1/1 [==============================] - 0s 13ms/step - loss: 6276.3242 - val_loss: 6219.7017
Epoch 1588/10000
1/1 [==============================] - 0s 13ms/step - loss: 5731.1152 - val_loss: 6795.0991
Epoch 1589/10000
1/1 [==============================] - 0s 13ms/step - loss: 6297.4214 - val_loss: 6125.5562
Epoch 1590/10000
1/1 [==============================] - 0s 13ms/step - loss: 6254.0527 - val_loss: 6269.3755
Epoch 1591/10000
1/1 [==============================] - 0s 13ms/step - loss: 6243.0918 - val_loss: 6251.4653
Epoch 1592/10000
1/1 [==============================] - 0s 13ms/step - loss: 6236.8960 - val_loss: 6279.7993
Epoch 1593/10000
1/1 [==============================] - 0s 14ms/step - loss: 6224.6626 - val_loss: 6274.0288
Epoch 1594/10000
1/1 [==============================] - 0s 14ms/step - loss: 6215.8198 - val_loss: 6287.7363
Epoch 1595/10000
1/1 [==============

In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[511.494](test_target) - [[439.3439]](prediction) = 72.1500974121094


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
0,-1.389631,-1.725654,1.865102,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-0.945398,-1.598407,-1.764646,-1.381098,-1.965308
1,-1.235264,-1.109019,1.025489,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.187517,-1.052322,-0.977404,-0.355590,-0.895551
2,-0.828641,-0.555899,0.724239,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.428730,-0.567038,-0.720813,0.222917,-0.224370
3,-0.123500,-0.064920,-0.343782,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.889078,-0.111646,-0.024492,1.148177,0.484341
4,0.376614,0.365297,-0.635537,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.626923,0.267535,0.557350,1.297877,0.730842
5,0.686805,0.736125,-0.977002,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.012054,0.572885,0.706370,0.959854,0.611879
6,1.071779,1.048943,-0.613273,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.824036,0.741306,0.858179,-1.264051,-0.192686
7,1.441839,1.305126,-1.045236,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,2.241557,1.747687,1.365455,-0.628085,1.450852


train_target:


,Sergipe - Consumo de Cimento (t)
0,222.063
1,221.797
2,270.225
3,319.406
4,391.414
5,404.787
6,480.282
7,579.482


test_input:


...

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
17,-0.858351,1.1315,1.051443,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.652049,0.766714,-1.279722,-1.105281,-0.602615


test_target:


,Sergipe - Consumo de Cimento (t)
17,511.494


1/1 [==============================] - 0s 34ms/step
Error: 72.1500974121094




[444.94317626953125,
 592.9867553710938,
 595.239990234375,
 624.2064208984375,
 581.4368896484375,
 576.1337890625,
 553.7530517578125,
 485.2795715332031,
 422.4762268066406,
 439.3439025878906]

In [23]:
display(targets)
display(predictions)

[646.0840000000001,
 590.339,
 633.1901428571429,
 592.0145142857143,
 552.6243428571429,
 473.789,
 372.21,
 409.903,
 526.396,
 511.494]

[444.94317626953125,
 592.9867553710938,
 595.239990234375,
 624.2064208984375,
 581.4368896484375,
 576.1337890625,
 553.7530517578125,
 485.2795715332031,
 422.4762268066406,
 439.3439025878906]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

83.80774680873324

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.15788819159888887